<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generate-Coarse-Grained-Model" data-toc-modified-id="Generate-Coarse-Grained-Model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate Coarse Grained Model</a></span></li><li><span><a href="#Generate-System" data-toc-modified-id="Generate-System-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generate System</a></span></li><li><span><a href="#Add-the-forces" data-toc-modified-id="Add-the-forces-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Add the forces</a></span></li><li><span><a href="#Set-up-the-simulation" data-toc-modified-id="Set-up-the-simulation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Set up the simulation</a></span></li></ul></div>

## Generate Coarse Grained Model

In [1]:
import open3SPN2
import ffAWSEM

In [2]:
#Fix the system (adds missing atoms)
fix=open3SPN2.fixPDB("1lmb.pdb")

In [3]:
#Create a table containing both the proteins and the DNA
complex_table=open3SPN2.pdb2table(fix)
complex_table.head()

,recname,serial,name,altLoc,resname,chainID,resSeq,iCode,x,y,z,occupancy,tempFactor,element,charge
serial,,,,,,,,,,,,,,,
1,ATOM,1,O5',,DA,1,1,,31.960001,-0.182000,33.011000,0,0,O,
2,ATOM,2,HO5',,DA,1,1,,31.833775,0.986762,32.860937,0,0,H,
3,ATOM,3,C5',,DA,1,1,,33.188000,-0.901000,32.936001,0,0,C,
4,ATOM,4,H5',,DA,1,1,,33.754478,-0.620150,33.953166,0,0,H,
5,ATOM,5,H5'',,DA,1,1,,33.918552,-0.451668,32.101724,0,0,H,


In [4]:
#Coarse Grain the system
dna_atoms=open3SPN2.DNA.CoarseGrain(complex_table)
protein_atoms=ffAWSEM.Protein.CoarseGrain(complex_table)

In [5]:
#Merge the models
import pandas
Coarse=pandas.concat([dna_atoms,protein_atoms],sort=False)
Coarse.index=range(len(Coarse))
Coarse.serial=list(Coarse.index)
Coarse.tail()

,recname,serial,name,altLoc,resname,chainID,resSeq,iCode,x,y,z,occupancy,tempFactor,element,charge,type,real_resname
1166,ATOM,1166,N,,NGP,4,92,,26.416338,-23.606008,4.960431,0,0,N,,NaN,SER
1167,ATOM,1167,H,,NGP,4,92,,26.274830,-22.881114,4.371276,0,0,H,,NaN,SER
1168,ATOM,1168,CA,,NGP,4,92,,27.344999,-24.565001,4.447000,0,0,C,,NaN,SER
1169,ATOM,1169,O,,NGP,4,92,,28.513999,-24.484999,6.607000,0,0,O,,NaN,SER
1170,ATOM,1170,CB,,NGP,4,92,,27.649000,-24.547999,2.924000,0,0,B,,NaN,SER


In [6]:
#Save the protein_sequence
from Bio.PDB.Polypeptide import three_to_one
_AWSEMresidues=['IPR','IGL','NGP']
protein_data=Coarse[Coarse.resname.isin(_AWSEMresidues)].copy()
resix = (protein_data.chainID + '_' + protein_data.resSeq.astype(str))
res_unique = resix.unique()
protein_data['resID'] = resix.replace(dict(zip(res_unique, range(len(res_unique)))))
protein_sequence=[r.iloc[0]['real_resname'] for i, r in protein_data.groupby('resID')]      
protein_sequence_one = [three_to_one(a) for a in protein_sequence]

with open('protein.seq','w+') as ps:
    ps.write(''.join(protein_sequence_one))

In [7]:
# Create a merged PDB
def writePDB(atoms,pdb_file):
    with open(pdb_file, 'w+') as pdb:
        for i, atom in atoms.iterrows():
            pdb_line = f'{atom.recname:<6}{atom.serial:>5} {atom["name"]:^4}{atom.altLoc:1}'+\
                       f'{atom.resname:<3} {atom.chainID:1}{atom.resSeq:>4}{atom.iCode:1}   '+\
                       f'{atom.x:>8.3f}{atom.y:>8.3f}{atom.z:>8.3f}' +\
                       f'{atom.occupancy:>6.2f}{atom.occupancy:>6.2f}'+' ' * 10 +\
                       f'{atom.element:>2}{atom.charge:>2}'
            assert len(pdb_line) == 80, f'An item in the atom table is longer than expected ({len(pdb_line)})\n{pdb_line}'
            pdb.write(pdb_line + '\n')
writePDB(Coarse,'clean.pdb')

## Generate System

In [8]:
#Create the merged system
import simtk.openmm
pdb=simtk.openmm.app.PDBFile('clean.pdb')
top=pdb.topology
coord=pdb.positions
forcefield=simtk.openmm.app.ForceField(ffAWSEM.xml,open3SPN2.xml)
s=forcefield.createSystem(top)

## Add the forces

In [9]:
dna=open3SPN2.DNA.fromCoarsePDB('clean.pdb')
with open('protein.seq') as ps:
    protein_sequence_one=ps.readlines()[0]
protein=ffAWSEM.Protein.fromCoarsePDB('clean.pdb',sequence=protein_sequence_one)
dna.periodic=False
protein.periodic=False

In [10]:
ffAWSEM.copy_parameter_files()

In [11]:
#Clear Forces from the system
keepCMMotionRemover=True
j=0
for i, f in enumerate(s.getForces()):
    if keepCMMotionRemover and i == 0 and f.__class__ == simtk.openmm.CMMotionRemover:
        # print('Kept ', f.__class__)
        j += 1
        continue
    else:
        # print('Removed ', f.__class__)
        s.removeForce(j)
if keepCMMotionRemover == False:
    assert len(s.getForces()) == 0, 'Not all the forces were removed'
else:
    assert len(s.getForces()) <= 1, 'Not all the forces were removed'
forces={}
for i in range(s.getNumForces()):
    force = s.getForce(i)
    force_name="CMMotionRemover"

#Add 3SPN2 forces
for force_name in open3SPN2.forces:
    print(force_name)
    force = open3SPN2.forces[force_name](dna)
    if force_name in ['BasePair','CrossStacking']:
        force.addForce(s)
    else:
        s.addForce(force)
    forces.update({force_name: force})
    
#Add AWSEM forces
openAWSEMforces = dict(Connectivity=ffAWSEM.functionTerms.basicTerms.con_term,
                       Chain=ffAWSEM.functionTerms.basicTerms.chain_term,
                       Chi=ffAWSEM.functionTerms.basicTerms.chi_term,
                       Excl=ffAWSEM.functionTerms.basicTerms.excl_term,
                       rama=ffAWSEM.functionTerms.basicTerms.rama_term,
                       rama_pro=ffAWSEM.functionTerms.basicTerms.rama_proline_term,
                       #rama_ss=ffAWSEM.functionTerms.basicTerms.rama_ssweight_term,
                       contact=ffAWSEM.functionTerms.contactTerms.contact_term,
                       beta1 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_1,
                       beta2 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_2,
                       beta3 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_3,
                       pap1 = ffAWSEM.functionTerms.hydrogenBondTerms.pap_term_1,
                       pap2 = ffAWSEM.functionTerms.hydrogenBondTerms.pap_term_2,
                      )
protein.setup_virtual_sites(s)
for force_name in openAWSEMforces:
    print(force_name)
    if force_name in ['contact']:
        force = openAWSEMforces[force_name](protein, withExclusion=False,periodic=False)
        print(force.getNumExclusions())
        open3SPN2.addNonBondedExclusions(dna,force)        
        print(force.getNumExclusions())
    
    else:
        force = openAWSEMforces[force_name](protein)
    s.addForce(force)
    forces.update({force_name: force})

#Add DNA-protein interaction forces
for force_name in open3SPN2.protein_dna_forces:
    print(force_name)
    force = open3SPN2.protein_dna_forces[force_name](dna,protein)
    s.addForce(force)
    forces.update({force_name: force})

Bond
Angle
Stacking
Dihedral
BasePair
CrossStacking
Exclusion
Electrostatics
Connectivity
Chain
Chi
Excl
rama
rama_pro
contact
Number of atom:  1171 Number of residue:  179
Contact cutoff  1.0 nm
NonbondedMethod:  1
0
639
beta1
beta_1 term ON
beta2
beta_2 term ON
beta3
beta_3 term ON
pap1
pap_1 term ON
No ssweight given, assume all zero
pap2
pap_2 term ON
No ssweight given, assume all zero
ExclusionProteinDNA
ElectrostaticsProteinDNA


## Set up the simulation

In [12]:
import numpy as np
temperature=300 * simtk.openmm.unit.kelvin
#platform_name='CUDA'

platform_name='OpenCL'

integrator = simtk.openmm.LangevinIntegrator(temperature, 1 / simtk.openmm.unit.picosecond, 2 * simtk.openmm.unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)
simulation = simtk.openmm.app.Simulation(top,s, integrator, platform)
simulation.context.setPositions(coord)
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print(energy)

-1406.7860107421875


In [13]:
#Obtain total energy

energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy',round(energy,6),energy_unit.get_symbol())

#Obtain detailed energy

energies = {}
for force_name, force in forces.items():
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

TotalEnergy -1406.786011 kJ/mol
Bond 0.0 kJ/mol
Angle 0.0 kJ/mol
Stacking 203.56601 kJ/mol
Dihedral -503.999969 kJ/mol
BasePair -284.232208 kJ/mol
CrossStacking -47.58614 kJ/mol
Exclusion 23.991552 kJ/mol
Electrostatics 23.268293 kJ/mol
Connectivity 1899.296875 kJ/mol
Chain 1899.296875 kJ/mol
Chi 1899.296875 kJ/mol
Excl 1899.296875 kJ/mol
rama -1363.522705 kJ/mol
rama_pro -1363.522705 kJ/mol
contact -1041.547729 kJ/mol
beta1 -601.593384 kJ/mol
beta2 -601.593384 kJ/mol
beta3 -601.593384 kJ/mol
pap1 0.0 kJ/mol
pap2 0.0 kJ/mol
ExclusionProteinDNA 296.033478 kJ/mol
ElectrostaticsProteinDNA -10.459808 kJ/mol


In [14]:
#Add simulation reporters
import sys
dcd_reporter=simtk.openmm.app.DCDReporter(f'output.dcd', 1000)
energy_reporter=simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,
                                                   potentialEnergy=True, temperature=True)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(energy_reporter)

In [ ]:
#Run simulation
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(10000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)"
1000,2.0000000000000013,-2821.62255859375,285.56926619310553
2000,3.999999999999781,-2839.069580078125,315.4594998778808
3000,5.999999999999561,-2805.5634765625,323.20843224380087
4000,7.999999999999341,-2801.427734375,314.2296292320282
5000,10.000000000000009,-2491.56884765625,305.8918233488531
6000,12.000000000000677,-2708.4228515625,293.2187270882386
